In [4]:
%%bash
sudo pip install nltk
sudo pip install twython
sudo pip install folium

The directory '/home/vagrant/.cache/pip/http' or its parent directory is not owned by the current user and the cache has been disabled. Please check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
The directory '/home/vagrant/.cache/pip' or its parent directory is not owned by the current user and caching wheels has been disabled. check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
The directory '/home/vagrant/.cache/pip/http' or its parent directory is not owned by the current user and the cache has been disabled. Please check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
The directory '/home/vagrant/.cache/pip' or its parent directory is not owned by the current user and caching wheels has been disabled. check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
The direct

In [5]:
import numpy as np
import pandas as pd
import nltk
import matplotlib.pyplot as plt
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from pylab import polyfit, poly1d
import folium
from folium.plugins import HeatMap
import math
from sklearn import linear_model, metrics
from sklearn.model_selection import cross_val_score, KFold, cross_val_predict

In [7]:
#part 3

df = pd.read_csv("boliga_zealand.csv").drop(['Index', '_1', 'Unnamed: 0'], axis=1)
df.head()

,address,zip_code,price,sell_date,sell_type,price_per_sq_m,no_rooms,housing_type,size_in_sq_m,year_of_construction,price_change_in_pct,lon,lat
0,"Ved Volden 5, 5. TV",1425 København K,4000000,23-05-2017,Alm. Salg,43956.0,3.0,Lejlighed,91.0,1938.0,0.0,12.593629,55.671769
1,"Rådhusstræde 4C, 1",1466 København K,4895000,18-05-2017,Alm. Salg,46619.0,3.0,Lejlighed,105.0,1796.0,0.0,12.573689,55.676839
2,"Store Kongensgade 112A, 3",1264 København K,250000,15-05-2017,Andet,1851.0,2.0,Lejlighed,135.0,1860.0,0.0,12.590441,55.687079
3,"Amaliegade 13G, 2",1256 København K,7375000,15-05-2017,Alm. Salg,75255.0,3.0,Lejlighed,98.0,1948.0,9.0,12.591287,55.683439
4,"Borgergade 144, 3. TH",1300 København K,5825000,10-05-2017,Alm. Salg,57673.0,3.0,Lejlighed,101.0,1854.0,3.0,12.588744,55.687623


In [8]:
dataset = df[['lon','lat','price']].dropna()

In [9]:
folium_map = folium.Map(location=[55.6831224,12.5693172], zoom_start=10)
heat_data = [(i.lat,i.lon,float(i.price)) for i in dataset.itertuples()]

In [10]:
HeatMap(heat_data, radius=8).add_to(folium_map)
folium_map.save('heatmap.html')

# Brønshøj virker til at være det sted som er billigst at bo hvis man samtidig ønsker at bo tæt på centrum af københavn
# Generelt ser det ud til at den vestlige del af København er dyrere end den østlige del

In [11]:
# Part 4
def calc_distance(lat1,lon1,lat2,lon2):
    R = 6371; # Radius of the earth in km
    
    dLat = math.radians(lat2-lat1)
    dLon = math.radians(lon2-lon1) 
    a = math.sin(dLat/2) * math.sin(dLat/2) + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dLon/2) * math.sin(dLon/2)
     
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = R * c # Distance in km
    return d

In [13]:
distance_df = df.dropna()
distance_df = distance_df.assign(distance_to_center=distance_df.apply(lambda row: calc_distance(row['lat'],row['lon'],55.676098, 12.568337),axis=1))

In [19]:
regr = linear_model.LinearRegression()

x_values= distance_df['distance_to_center'].astype(float).values.reshape(-1,1)
y_values = distance_df['price'].astype(int).values.reshape(-1,1)
z_values = distance_df['size_in_sq_m'].astype(float).values.reshape(-1,1)

In [20]:
folds = KFold(n_splits=10)

metrics_df = pd.DataFrame(columns=['Coefficients','Intercept','MAE','MSE','Pearson'])

metrics_list = []

for train_index, test_index in folds.split(x_values, y_values, z_values):
    
    xz_values = np.stack([x_values,y_values], axis=1).reshape(-1,2)
    
    xz_train, xz_test = xz_values[train_index], xz_values[test_index]
    
    y_train, y_test = y_values[train_index], y_values[test_index]
    
    regr = linear_model.LinearRegression()
    
    regr.fit(xz_train, y_train)
    
    prediction = regr.predict(xz_test)
    
    coef = str(regr.coef_)
    intercept = str(regr.intercept_)
    MAE = str(metrics.mean_absolute_error(y_test,prediction))
    MSE = str(math.sqrt(metrics.mean_squared_error(y_test,prediction)))
    Pearson = str(metrics.r2_score(y_test, prediction))
    

    metrics_list.append({'Coefficients':coef,'Intercept':intercept,'MAE':MAE,'MSE':MSE,'Pearson':Pearson})

metrics_df = metrics_df.append(metrics_list)
    
metrics_df

,Coefficients,Intercept,MAE,MSE,Pearson
0,[[ 1.84427961e-10 1.00000000e+00]],[ -9.31322575e-09],4.17963059848e-09,6.259016307107142e-09,1.0
1,[[ -1.05326790e-10 1.00000000e+00]],[ 1.25728548e-08],6.50239653777e-09,1.0952563348172403e-08,1.0
2,[[ -2.28808802e-10 1.00000000e+00]],[ 3.44589353e-08],1.5896846022e-08,2.167169943934918e-08,1.0
3,[[ 4.59473966e-11 1.00000000e+00]],[ -1.07102096e-08],4.98630256823e-09,6.765708810875755e-09,1.0
4,[[ 2.70797960e-10 1.00000000e+00]],[ -4.33064997e-08],1.72623206831e-08,2.602388574984423e-08,1.0
5,[[ 3.81226935e-11 1.00000000e+00]],[ -7.91624188e-09],3.14056306685e-09,5.419892787913968e-09,1.0
6,[[ -4.55782703e-12 1.00000000e+00]],[ -6.98491931e-09],5.78244744593e-09,1.05570979589608e-08,1.0
7,[[ -2.51734341e-10 1.00000000e+00]],[ 3.77185643e-08],1.64522812156e-08,3.127467452907649e-08,1.0
8,[[ -4.43183252e-11 1.00000000e+00]],[ -1.16415322e-08],6.73989159973e-09,1.1608137177326947e-08,1.0
9,[[ -1.31101893e-10 1.00000000e+00]],[ 3.58559191e-08],1.26752253699e-08,2.1408925733916345e-08,1.0
